In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pylab as plt

In [2]:
train = pd.read_json('./data/processed/train.json') 
test = pd.read_json('./data/processed/test.json')
#sample_submission = pd.read_csv('./data/processed/sample_submission.csv')

In [3]:
print(len(train))
display(train.head())

1604


,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [4]:
print(len(test))
display(test.head())

8424


,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",5941774d,34.966400
1,"[-26.058969497680664, -26.058969497680664, -26...","[-25.754207611083984, -25.754207611083984, -25...",4023181e,32.615072
2,"[-14.14109992980957, -15.064241409301758, -17....","[-14.74563980102539, -14.590410232543945, -14....",b20200e4,37.505433
3,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",e7f018bb,34.473900
4,"[-23.37459373474121, -26.02718162536621, -28.1...","[-25.72234344482422, -27.011577606201172, -23....",4371c8c3,43.918874


In [5]:
train['inc_angle2'] = train['inc_angle']
train['inc_angle2'] = train['inc_angle'].where(train['inc_angle'] != 'na', 0)

In [6]:
train_id = train['id'].values
test_id = test['id'].values
train_inc_angle = np.array(train['inc_angle2'].values, dtype='float32').reshape(len(train), 1)
test_inc_angle = np.array(test['inc_angle'].values, dtype='float32').reshape(len(test), 1)
train_band_1 = np.array(list(map(list, train['band_1'].values)), dtype='float32').reshape(len(train), 75, 75, 1)
train_band_2 = np.array(list(map(list, train['band_2'].values)), dtype='float32').reshape(len(train), 75, 75, 1)
test_band_1 = np.array(list(map(list, test['band_1'].values)), dtype='float32').reshape(len(test), 75, 75, 1)
test_band_2 = np.array(list(map(list, test['band_2'].values)), dtype='float32').reshape(len(test), 75, 75, 1)

In [7]:
tmp = np.vstack([train_inc_angle, test_inc_angle])
tmp_max = tmp.max()
tmp_min = tmp.min()

train_inc_angle2 = (train_inc_angle - tmp_min) / (tmp_max - tmp_min) 
test_inc_angle2 = (test_inc_angle - tmp_min) / (tmp_max - tmp_min)

In [8]:
tmp = np.vstack([train_band_1, test_band_1])
tmp_max = tmp.max()
tmp_min = tmp.min()

train_band_1_2 = (train_band_1 - tmp_min) / (tmp_max - tmp_min)
test_band_1_2 = (test_band_1 - tmp_min) / (tmp_max - tmp_min)

In [9]:
tmp = np.vstack([train_band_2, test_band_2])
tmp_max = tmp.max()
tmp_min = tmp.min()

train_band_2_2 = (train_band_2 - tmp_min) / (tmp_max - tmp_min)
test_band_2_2 = (test_band_2 - tmp_min) / (tmp_max - tmp_min)

In [10]:
train_is_iceberg2 = np.eye(2)[train["is_iceberg"].values].astype('float32')

In [11]:
N = 1200
train_band_1_2, varid_band_1_2 = np.split(train_band_1_2, [N])
train_band_2_2, varid_band_2_2 = np.split(train_band_2_2, [N])
train_inc_angle2, varid_inc_angle2 = np.split(train_inc_angle2, [N])
train_is_iceberg2, varid_is_iceberg2 = np.split(train_is_iceberg2, [N])

In [13]:
import tensorflow as tf
from tqdm import tqdm

In [15]:
xb1_ = tf.placeholder(tf.float32, shape=(None, 75, 75, 1)) # x: band_1
xb2_ = tf.placeholder(tf.float32, shape=(None, 75, 75, 1)) # x: band_2
xia_ = tf.placeholder(tf.float32, shape=(None, 1)) # x: inc_angle
y_ = tf.placeholder(tf.float32, shape=(None, 2)) # y: is_iceberg

xb1_conv1_features = 30
xb1_max_pool_size1 = 2
xb1_conv1_w = tf.Variable(tf.truncated_normal([5, 5, 1, xb1_conv1_features], stddev=0.1), dtype=tf.float32)
xb1_conv1_b = tf.Variable(tf.constant(0.1, shape=[xb1_conv1_features]), dtype=tf.float32)
xb1_conv1_c2 = tf.nn.conv2d(xb1_, xb1_conv1_w, strides=[1,1,1,1], padding='SAME')
xb1_conv1_relu = tf.nn.relu(xb1_conv1_c2+xb1_conv1_b)
xb1_conv1_mp = tf.nn.max_pool(xb1_conv1_relu, ksize=[1, xb1_max_pool_size1, xb1_max_pool_size1, 1], strides=[1, xb1_max_pool_size1, xb1_max_pool_size1, 1], padding='SAME')
xb1_s = xb1_conv1_mp.get_shape().as_list()
xb1_conv_result = tf.reshape(xb1_conv1_mp, [-1, xb1_s[1]*xb1_s[2]*xb1_s[3]])

xb2_conv1_features = 30
xb2_max_pool_size1 = 2
xb2_conv1_w = tf.Variable(tf.truncated_normal([5, 5, 1, xb2_conv1_features], stddev=0.1), dtype=tf.float32)
xb2_conv1_b = tf.Variable(tf.constant(0.1, shape=[xb2_conv1_features]), dtype=tf.float32)
xb2_conv1_c2 = tf.nn.conv2d(xb2_, xb2_conv1_w, strides=[1,1,1,1], padding='SAME')
xb2_conv1_relu = tf.nn.relu(xb2_conv1_c2+xb2_conv1_b)
xb2_conv1_mp = tf.nn.max_pool(xb2_conv1_relu, ksize=[1, xb2_max_pool_size1, xb2_max_pool_size1, 1], strides=[1, xb2_max_pool_size1, xb2_max_pool_size1, 1], padding='SAME')
xb2_s = xb1_conv1_mp.get_shape().as_list()
xb2_conv_result = tf.reshape(xb2_conv1_mp, [-1, xb2_s[1]*xb2_s[2]*xb2_s[3]])

xia_fc1_features = 50
xia_fc1_w = tf.Variable(tf.truncated_normal([1, xia_fc1_features], stddev=0.1), dtype=tf.float32)
xia_fc1_b = tf.Variable(tf.constant(0.1, shape=[xia_fc1_features]), dtype=tf.float32)
xia_fc1 = tf.nn.relu(tf.matmul(xia_, xia_fc1_w)+xia_fc1_b)

result_concat = tf.concat([xb1_conv_result, xb2_conv_result, xia_fc1], axis=1)

fc_features = 100
fc1_w = tf.Variable(tf.truncated_normal([result_concat.shape[1].value, fc_features], stddev=0.1), dtype=tf.float32)
fc1_b = tf.Variable(tf.constant(0.1, shape=[fc_features]), dtype=tf.float32)
fc1_m = tf.matmul(result_concat, fc1_w)+fc1_b
fc1 = fc1_m*tf.nn.sigmoid(fc1_m)

fc2_w = tf.Variable(tf.truncated_normal([fc_features, 2], stddev=0.1), dtype=tf.float32)
fc2_b = tf.Variable(tf.constant(0.1, shape=[2]), dtype=tf.float32)
y = tf.matmul(fc1, fc2_w)+fc2_b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
prob = tf.nn.softmax(y)

EPOCH_NUM = 100
BATCH_SIZE = 200

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(EPOCH_NUM):
    perm = np.random.permutation(N)
    loss  = 0
    for i in range(0, N, BATCH_SIZE):
        batch_xb1 = train_band_1_2[perm[i:i+BATCH_SIZE]]
        batch_xb2 = train_band_2_2[perm[i:i+BATCH_SIZE]]
        batch_xia = train_inc_angle2[perm[i:i+BATCH_SIZE]]
        batch_y = train_is_iceberg2[perm[i:i+BATCH_SIZE]]
        train_step.run(session=sess, feed_dict={xb1_: batch_xb1, xb2_: batch_xb2, xia_: batch_xia, y_: batch_y})
    train_accuracy = accuracy.eval(session=sess, feed_dict={xb1_: train_band_1_2, xb2_: train_band_2_2, xia_: train_inc_angle2, y_: train_is_iceberg2})
    test_accuracy = accuracy.eval(session=sess, feed_dict={xb1_: varid_band_1_2, xb2_: varid_band_2_2, xia_: varid_inc_angle2, y_: varid_is_iceberg2})
    if (epoch+1) % 10 == 0:
        print("epoch:\t{}\tloss:\t{}\ttrain acc:\t{}\ttest acc:\t{}".format(epoch+1, loss, train_accuracy, test_accuracy))

epoch:	10	loss:	0	train acc:	0.6633333563804626	test acc:	0.6212871074676514
epoch:	20	loss:	0	train acc:	0.7333333492279053	test acc:	0.6732673048973083
epoch:	30	loss:	0	train acc:	0.7925000190734863	test acc:	0.6732673048973083
epoch:	40	loss:	0	train acc:	0.9024999737739563	test acc:	0.7821782231330872
epoch:	50	loss:	0	train acc:	0.9300000071525574	test acc:	0.7920792102813721
epoch:	60	loss:	0	train acc:	0.9208333492279053	test acc:	0.8316831588745117
epoch:	70	loss:	0	train acc:	0.9674999713897705	test acc:	0.8193069100379944
epoch:	80	loss:	0	train acc:	0.9758333563804626	test acc:	0.8415841460227966
epoch:	90	loss:	0	train acc:	0.9900000095367432	test acc:	0.853960394859314
epoch:	100	loss:	0	train acc:	0.9891666769981384	test acc:	0.8465346693992615


In [20]:
p = prob.eval(session=sess, feed_dict={xb1_: varid_band_1_2, xb2_: varid_band_2_2, xia_: varid_inc_angle2})
p[:, 1][:100]

array([0.7158221 , 0.19485727, 0.77252895, 0.55088705, 0.8516039 ,
       0.22196984, 0.83809304, 0.3239814 , 0.03360997, 0.3305467 ,
       0.6967359 , 0.14113627, 0.9936469 , 0.19300793, 0.00592783,
       0.06695471, 0.56661606, 0.26381668, 0.02489205, 0.62879974,
       0.9628189 , 0.75322825, 0.00584668, 0.74227095, 0.31670472,
       0.1593704 , 0.03078315, 0.09333576, 0.01285369, 0.17544198,
       0.2585224 , 0.0385753 , 0.03175983, 0.08054496, 0.0539799 ,
       0.6009393 , 0.47134802, 0.54488593, 0.23239134, 0.6594467 ,
       0.00477477, 0.25164202, 0.04261281, 0.06691042, 0.9172952 ,
       0.5361519 , 0.9726777 , 0.00383775, 0.3702118 , 0.00273449,
       0.9120252 , 0.03584305, 0.50983876, 0.8166622 , 0.03822677,
       0.00904701, 0.17514555, 0.12455195, 0.06210341, 0.97703904,
       0.666223  , 0.00873818, 0.10843845, 0.093592  , 0.83911395,
       0.42779815, 0.3690727 , 0.37122962, 0.0191766 , 0.1493067 ,
       0.82803327, 0.00512316, 0.49745604, 0.13680136, 0.91536

In [24]:
pre = np.array([], dtype=np.float32)

tmp_size = 1000
for i in tqdm(range(0, len(test), tmp_size)):
    batch_xb1 = test_band_1_2[i:i+tmp_size]
    batch_xb2 = test_band_2_2[i:i+tmp_size]
    batch_xia = test_inc_angle2[i:i+tmp_size]
    p = prob.eval(session=sess, feed_dict={xb1_: batch_xb1, xb2_: batch_xb2, xia_: batch_xia})
    pre = np.hstack([pre, p[:,1]])

100%|██████████| 9/9 [00:00<00:00, 22.56it/s]


In [27]:
test_id.shape, pre.shape

((8424,), (8424,))